# BB84 protocol.

https://en.wikipedia.org/wiki/BB84

In [225]:
from qiskit import *
from qiskit.visualization import *
import random 

def measure_and_count(circuit, N):
    for i in range(N):
        circuit.measure(i, i)
    sim = Aer.get_backend('aer_simulator')
    job = execute(circuit, sim)
    result = job.result()
    counts = result.get_counts()
    return counts


### Objective 1; Transfer 10 bits data from Alice to bob

In [226]:
KeyRandom = random.choices([0,1], k=15)
KeyToBeTransimtted = "".join([str(a) for a in KeyRandom])

N = len(KeyToBeTransimtted)
BobFullKeyReceived = False 
BobsKey = [-1]*N
iterations = 0 

for i in range(4*N):
    QuantumBB84Circuit = QuantumCircuit(N,N)
    for idx, i in enumerate(KeyToBeTransimtted):
        if i == "1":
            QuantumBB84Circuit.x(idx)

    QuantumBB84Circuit.barrier()

    # Alice choses a basis randomly.

    alice_basis_choice = random.choices([0,1], k=N)
    for idx, c in enumerate(alice_basis_choice):
        if c == 1:
            QuantumBB84Circuit.h(idx)

    QuantumBB84Circuit.barrier()

    # Bob choses a basis randomly.
    bob_basis_choice = random.choices([0,1], k=N)
    for idx, c in enumerate(bob_basis_choice):
        if c == 1:
            QuantumBB84Circuit.h(idx)
    QuantumBB84Circuit.barrier()
    QuantumBB84Circuit.draw()
    z = measure_and_count(QuantumBB84Circuit, N)
    
    ## publicly share alice and bob basis. 

    reference = list(z.keys())

    reference = [k[::-1] for k in reference]

    def get_all_values_at_index(reference, bit_no): 
        ans = reference[0][bit_no]

        for i in range(1, len(reference)):
            if ans != reference[i][bit_no]:
                return -1
        
        return ans 

    for idx, (alice_choice, bob_choice) in enumerate(zip(alice_basis_choice, bob_basis_choice)):
        if alice_choice == bob_choice:
            ans = get_all_values_at_index(reference, idx)
            if BobsKey[idx] == -1 and ans != -1:
                BobsKey[idx] = ans

    iterations += 1

    if -1 not in BobsKey:
        break

print("Alice key =>", KeyToBeTransimtted, "\nBobs  key =>", "".join(BobsKey), '\nretrieved in' , iterations, 'iterations')


Alice key => 100101110111111 
Bobs  key => 100101110111111 
retrieved in 4 iterations
